In [1]:
from pathlib import Path
import numpy as np
import xarray as xr
import easygems.healpix as egh
import uxarray as ux
import healpy as hp
import cartopy.crs as ccrs
import cartopy.feature as cf
import matplotlib.pyplot as plt


In [ ]:
egh.__all__

In [7]:
# loc = Path("/glade/derecho/scratch/brianpm/healpix/")
# loc = Path("/glade/derecho/scratch/digital-earths-hackathon/mpas_DYAMOND3/1hr")
# loc = Path('/glade/derecho/scratch/digital-earths-hackathon/mpas_DYAMOND3/v4')
loc = Path("/glade/derecho/scratch/digital-earths-hackathon/mpas_DYAMOND1")
test_fil_name = "DYAMOND1_history_to_hp"
for zoom in range(1, 11):
    print(f"------ zoom = {zoom} ---------")
    zfil = loc / f"{test_fil_name}{zoom}.zarr"
    if zfil.exists():
        ds = xr.open_dataset(zfil)
        
        # print(f"zoom = {zoom}, exist: {zfil.exists()}, {np.sum(np.isnan(ds['w']))}")

------ zoom = 1 ---------
------ zoom = 2 ---------
------ zoom = 3 ---------
------ zoom = 4 ---------
------ zoom = 5 ---------
------ zoom = 6 ---------
------ zoom = 7 ---------
------ zoom = 8 ---------
------ zoom = 9 ---------
------ zoom = 10 ---------


In [8]:
z = 5
zfil = loc / f"{test_fil_name}{z}.zarr"
if zfil.exists():
    ds = xr.open_dataset(zfil)
    uxds = ux.UxDataset.from_healpix(ds)
else:
    print("ERROR!")


In [12]:
ds

<xarray.Dataset> Size: 3GB
Dimensions:                 (time: 48, cell: 12288, nVertLevels: 75,
                             nSoilLevels: 4, nVertLevelsP1: 76)
Coordinates:
  * time                    (time) datetime64[ns] 384B 2016-08-01 ... 2016-08...
Dimensions without coordinates: cell, nVertLevels, nSoilLevels, nVertLevelsP1
Data variables: (12/24)
    xice                    (time, cell) float32 2MB ...
    relhum                  (time, nVertLevels, cell) float32 177MB ...
    vegfra                  (time, cell) float32 2MB ...
    pressure                (time, nVertLevels, cell) float32 177MB ...
    nr                      (time, nVertLevels, cell) float32 177MB ...
    sst                     (time, cell) float32 2MB ...
    ...                      ...
    qi                      (time, nVertLevels, cell) float32 177MB ...
    smois                   (time, nSoilLevels, cell) float32 9MB ...
    qv                      (time, nVertLevels, cell) float32 177MB ...
    temperature             (time, nVertLevels, cell) float32 177MB ...
    qs                      (time, nVertLevels, cell) float32 177MB ...
    uReconstructZonal       (time, nVertLevels, cell) float32 177MB ...
Attributes: (12/114)
    model_name:                        mpas
    core_name:                         atmosphere
    source:                            MPAS
    Conventions:                       MPAS
    git_version:                       v6.1-dirty
    on_a_sphere:                       YES
    ...                                ...
    config_oml_relaxation_time:        864000.0
    config_sounding_interval:          none
    file_id:                           1jssg9g7bm
    NCO:                               netCDF Operators version 4.9.5 (Homepa...
    history:                           Wed Mar 24 19:39:08 2021: ncks -A -v x...
    history_of_appended_files:         Wed Mar 24 19:39:08 2021: Appended fil...

In [ ]:
data_var = 'lwupt'
uxds[data_var].isel(time=1).plot(cmap="inferno", projection=ccrs.Mollweide(), title="Remapped Data")


In [ ]:
ds['lh_tavg'].max()

In [ ]:
xsample = ds['ivt'].squeeze()

print(f"Size of xsample: {xsample.size}, Units: {xsample.attrs.get("units", "NONE")}, Minimum: {xsample.min().item()}, Maximum: {xsample.max().item()}")

In [ ]:
egh.healpix_contour(xsample)

In [ ]:
# An improved contour method:
import numpy as np
from cartopy.util import add_cyclic_point

def healpix_contour_cyc(
    var, dpi=None, ax=None, method="linear", nest=True, add_coastlines=True, **kwargs
):
    if ax is None:
        ax = egh.get_current_geoaxis(add_coastlines=add_coastlines)
    fig = ax.get_figure()

    if dpi is not None:
        fig.set_dpi(dpi)

    _, _, nx, ny = np.array(ax.bbox.bounds, dtype=int)

    xlims = ax.get_xlim()
    ylims = ax.get_ylim()

    im = egh.healpix_resample(var, xlims, ylims, nx, ny, ax.projection, method, nest)
    # Add cyclic point
    cyclic_data, cyclic_lon = add_cyclic_point(im, coord=im.x)

    return ax.contour(cyclic_lon, im.y, cyclic_data, **kwargs)


def healpix_contourf_cyc(
    var, dpi=None, ax=None, method="linear", nest=True, add_coastlines=True, **kwargs
):
    if ax is None:
        ax = egh.get_current_geoaxis(add_coastlines=add_coastlines)
    fig = ax.get_figure()

    if dpi is not None:
        fig.set_dpi(dpi)

    _, _, nx, ny = np.array(ax.bbox.bounds, dtype=int)

    xlims = ax.get_xlim()
    ylims = ax.get_ylim()

    im = egh.healpix_resample(var, xlims, ylims, nx, ny, ax.projection, method, nest)
    # Add cyclic point
    cyclic_data, cyclic_lon = add_cyclic_point(im, coord=im.x)

    return ax.contourf(cyclic_lon, im.y, cyclic_data, **kwargs)



In [ ]:
healpix_contourf_cyc(xsample)
# add_cyclic_point?

print(arr)
print(arr.max())

In [ ]:
myaxis = egh.create_geoaxis(add_coastlines=True)

myaxis.get_xlim()
myaxis.get_ylim()

In [ ]:

egh.isel_extent(egh.attach_coords(ds), (10, 90, -45, 45))

In [ ]:
# ds.get("cell")

egh.get_nest(ds)


# dsll = egh.attach_coords(ds)

In [ ]:
# egh.isel_extent(dsll, (10, 90, -45, 45))
lons, lats = egh.healpix.pix2ang(egh.get_nside(ds), ds.get('cell'), nest=egh.get_nest(ds), lonlat=True)

In [ ]:
egh.get_nside(ds)

In [ ]:
egh.healpix.pix2ang(np.array([1, 2, 4, 8]), 11, nest=True, lonlat=True)

In [ ]:
2**20

In [ ]:
import xarray as xr

In [ ]:
ds = xr.open_dataset("/glade/work/brianpm/model_data/aquaplanet_ozone_hightop_c160920.nc")
ds

In [ ]:
chunks = {dim:-1 for dim in ds.dims}
chunks['time'] = 10
chunks['lon'] = 10000
chunks

## Speed Test for getting zarr time


In [2]:
%%time
ds = xr.open_dataset("/glade/derecho/scratch/digital-earths-hackathon/mpas_DYAMOND1/DYAMOND1_history_to_hp10.zarr")

CPU times: user 377 ms, sys: 26.3 ms, total: 403 ms
Wall time: 1.15 s


In [3]:
ds['time']

<xarray.DataArray 'time' (time: 34)> Size: 272B
array(['2016-08-01T00:00:00.000000000', '2016-08-01T03:00:00.000000000',
       '2016-08-01T06:00:00.000000000', '2016-08-01T09:00:00.000000000',
       '2016-08-01T12:00:00.000000000', '2016-08-01T15:00:00.000000000',
       '2016-08-01T18:00:00.000000000', '2016-08-01T21:00:00.000000000',
       '2016-08-02T00:00:00.000000000', '2016-08-02T03:00:00.000000000',
       '2016-08-02T06:00:00.000000000', '2016-08-02T09:00:00.000000000',
       '2016-08-02T12:00:00.000000000', '2016-08-02T15:00:00.000000000',
       '2016-08-02T18:00:00.000000000', '2016-08-02T21:00:00.000000000',
       '2016-08-03T00:00:00.000000000', '2016-08-03T03:00:00.000000000',
       '2016-08-03T06:00:00.000000000', '2016-08-03T09:00:00.000000000',
       '2016-08-03T12:00:00.000000000', '2016-08-03T15:00:00.000000000',
       '2016-08-03T18:00:00.000000000', '2016-08-03T21:00:00.000000000',
       '2016-08-04T00:00:00.000000000', '2016-08-04T03:00:00.000000000',
       '2016-08-04T06:00:00.000000000', '2016-08-04T09:00:00.000000000',
       '2016-08-04T12:00:00.000000000', '2016-08-04T15:00:00.000000000',
       '2016-08-04T18:00:00.000000000', '2016-08-04T21:00:00.000000000',
       '2016-08-05T00:00:00.000000000', '2016-08-05T03:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 272B 2016-08-01 ... 2016-08-05T03:00:00

In [42]:
# get an example file that's already included:
import pandas as pd
def pre_proc_mpas_file(datafil, meshfil):
    ds_mpas = xr.open_dataset(datafil, engine='netcdf4', mask_and_scale=True, chunks={'Time': 'auto'})
    ref_date = '2000-01-01 00:00:00'  # Or any other suitable fixed date
    time_str = ds_mpas.xtime.astype(str).values.astype('U').ravel()
    time_str = [x.strip() for x in time_str]
    time_str = [x.replace("_", " ") for x in time_str]
    if isinstance(time_str, np.ndarray) or isinstance(time_str, list):
        time_str = "".join(time_str)
    time_coord = pd.to_datetime(time_str)
    hours_since = (time_coord - pd.Timestamp(ref_date)) / pd.Timedelta('1h')
    if isinstance(hours_since, xr.DataArray):
        hours_since = hours_since.values
    elif isinstance(hours_since, float):
        hours_since = np.array([hours_since,])
    time_var = xr.DataArray(
        hours_since,
        dims='Time',
        name='time',
        attrs={'long_name': 'time', 
               'axis': 'T',
               'reference_date': ref_date},
               )
    time_var.encoding = {
        'dtype': 'float64',
        'units': f'hours since {ref_date}',
        'calendar': 'standard',
        '_FillValue': None
    }    
    ds_mpas_new = ds_mpas.assign_coords(time=('Time', time_var.data))
    ds_mpas_new = ds_mpas_new.swap_dims({"Time":"time"})
    ds_mpas_new['time'].attrs = time_var.attrs
    s64_vars = [var for var in ds_mpas_new.variables if ds_mpas_new[var].dtype == 'S64']
    ds_mpas_clean = ds_mpas_new.drop_vars(s64_vars)
    ds_mpas_clean = ds_mpas_clean.drop_vars(['xtime', 'xtime_old'], errors='ignore')
    return ds_mpas_clean

eds = pre_proc_mpas_file("/glade/campaign/mmm/wmr/fjudt/projects/dyamond_1/3.75km/history.2016-08-05_03.00.00.nc", None)

eds_time = eds['time']
eds_time.attrs.update({
    'units': 'hours since 2000-01-01 00:00:00',
    'calendar': 'standard'
})
eds_time = xr.decode_cf(eds_time.to_dataset())

ValueError: cannot create a Dataset from a DataArray with the same name as one of its coordinates

In [ ]:

eds = xr.decode_cf(eds)

In [36]:
any(eds['time'][0] == ds['time'])

True

In [38]:
ds['time'].values

array(['2016-08-01T00:00:00.000000000', '2016-08-01T03:00:00.000000000',
       '2016-08-01T06:00:00.000000000', '2016-08-01T09:00:00.000000000',
       '2016-08-01T12:00:00.000000000', '2016-08-01T15:00:00.000000000',
       '2016-08-01T18:00:00.000000000', '2016-08-01T21:00:00.000000000',
       '2016-08-02T00:00:00.000000000', '2016-08-02T03:00:00.000000000',
       '2016-08-02T06:00:00.000000000', '2016-08-02T09:00:00.000000000',
       '2016-08-02T12:00:00.000000000', '2016-08-02T15:00:00.000000000',
       '2016-08-02T18:00:00.000000000', '2016-08-02T21:00:00.000000000',
       '2016-08-03T00:00:00.000000000', '2016-08-03T03:00:00.000000000',
       '2016-08-03T06:00:00.000000000', '2016-08-03T09:00:00.000000000',
       '2016-08-03T12:00:00.000000000', '2016-08-03T15:00:00.000000000',
       '2016-08-03T18:00:00.000000000', '2016-08-03T21:00:00.000000000',
       '2016-08-04T00:00:00.000000000', '2016-08-04T03:00:00.000000000',
       '2016-08-04T06:00:00.000000000', '2016-08-04